In [1]:
from utilities import get_all_species, get_compositional_features
import os

import torch
import ase.io
import numpy as np
from multiprocessing import cpu_count
from pathos.multiprocessing import ProcessingPool as Pool
from tqdm import tqdm
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader, DataListLoader
from torch import nn
import torch.nn.functional as F
from sklearn.linear_model import Ridge
from utilities import ModelKeeper
import time
from scipy.spatial.transform import Rotation
from torch.optim.lr_scheduler import LambdaLR
import sys
import copy
import inspect
import yaml
from torch_geometric.nn import DataParallel

from molecule import Molecule, batch_to_dict
from hypers import Hypers
from pet import PET
from utilities import FullLogger
from utilities import get_rmse, get_mae, get_relative_rmse, get_loss
from analysis import get_structural_batch_size, convert_atomic_throughput

In [2]:
EPSILON = 1e-10
STRUCTURES_PATH = '../datasets/coll/coll_v1.2_AE_test.xyz'
DEFAULT_HYPERS_PATH = 'default_hypers.yaml'
SP_HYPERS_PATH = 'sp_default_hypers.yaml'
NUM_MOLECULES = 4

In [3]:
hypers = Hypers()
hypers.load_from_file(DEFAULT_HYPERS_PATH)

hypers_only_length = copy.deepcopy(hypers)
hypers_only_length.USE_ONLY_LENGTH = True

In [4]:
structures = ase.io.read(STRUCTURES_PATH, index = f':{NUM_MOLECULES}')
all_species = get_all_species(structures)
molecules = [Molecule(structure, hypers.R_CUT, hypers.USE_ADDITIONAL_SCALAR_ATTRIBUTES, hypers.USE_FORCES) for structure in tqdm(structures)]

max_nums = [molecule.get_max_num() for molecule in molecules]
max_num = np.max(max_nums)
graphs = [molecule.get_graph(max_num, all_species) for molecule in tqdm(molecules)]
loader = DataLoader(graphs, batch_size=10, shuffle=False)

100%|███████████████████████████████████████████| 4/4 [00:00<00:00, 1307.04it/s]


In [5]:
def get_model(hypers):
    add_tokens = []
    for _ in range(hypers.N_GNN_LAYERS - 1):
        add_tokens.append(hypers.ADD_TOKEN_FIRST)
    add_tokens.append(hypers.ADD_TOKEN_SECOND)

    model = PET(hypers, hypers.TRANSFORMER_D_MODEL, hypers.TRANSFORMER_N_HEAD,
                           hypers.TRANSFORMER_DIM_FEEDFORWARD, hypers.N_TRANS_LAYERS, 
                           0.0, len(all_species), 
                           hypers.N_GNN_LAYERS, hypers.HEAD_N_NEURONS, hypers.TRANSFORMERS_CENTRAL_SPECIFIC, hypers.HEADS_CENTRAL_SPECIFIC, 
                           add_tokens).cuda()

    model.augmentation = False
    return model

model = get_model(hypers)
model_only_length = get_model(hypers_only_length)

In [6]:
def compute_for_random_rotation(model, is_sp = False):
    structures = ase.io.read(STRUCTURES_PATH, index = f':{NUM_MOLECULES}')
    
    rotation = Rotation.random(1).as_matrix()[0]
    for struc in structures:
        struc.positions = np.dot(struc.positions, rotation)
    
    all_species = get_all_species(structures)
    molecules = [Molecule(structure, hypers.R_CUT, hypers.USE_ADDITIONAL_SCALAR_ATTRIBUTES, hypers.USE_FORCES) for structure in structures]

    max_nums = [molecule.get_max_num() for molecule in molecules]
    max_num = np.max(max_nums)
    graphs = [molecule.get_graph(max_num, all_species) for molecule in molecules]
    loader = DataLoader(graphs, batch_size=1, shuffle=False)
    
    torch.set_printoptions(precision=10)
    result = []
    for batch in loader:
        batch.cuda()
        if is_sp:
            _, _, _, predictions_energies, _, _, _ = model(batch)
        else:
            predictions_energies, _, _, _ = model(batch)
        result.append(predictions_energies)
    result = torch.cat(result, dim = 0)
    print(result)
    
    

# Normal model is not invariant with respect to rotations:

In [7]:
        
for _ in range(10):
    compute_for_random_rotation(model)

tensor([ 9.2439546585,  3.6752846241, 23.5622901917,  5.5260915756],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 9.7120895386,  3.9531302452, 25.6354007721,  5.7538542747],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 9.2407379150,  3.8272337914, 24.4651470184,  5.4261684418],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 9.1754608154,  3.5353443623, 23.8813972473,  5.6925954819],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 9.5363912582,  3.5306556225, 23.2859497070,  5.7522954941],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 8.8836431503,  3.6675195694, 24.1495990753,  5.2963962555],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 9.0820541382,  3.7650759220, 25.1997337341,  5.3429045677],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 9.3361873627,  3.6924219131, 24.1268119812,  5.5006227493],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([10.0773077011,  3.6509943008, 25.5265960693,  6.24449062

# Only length model is invariant with respect to rotations

In [8]:
for _ in range(10):
    compute_for_random_rotation(model_only_length)

tensor([15.1958856583,  5.3312802315, 38.9591522217,  8.5581979752],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([15.1958847046,  5.3312802315, 38.9591522217,  8.5581979752],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([15.1958856583,  5.3312802315, 38.9591522217,  8.5581979752],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([15.1958856583,  5.3312802315, 38.9591484070,  8.5581979752],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([15.1958847046,  5.3312802315, 38.9591522217,  8.5581979752],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([15.1958847046,  5.3312802315, 38.9591484070,  8.5581989288],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([15.1958856583,  5.3312797546, 38.9591484070,  8.5581979752],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([15.1958847046,  5.3312807083, 38.9591484070,  8.5581979752],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([15.1958856583,  5.3312807083, 38.9591484070,  8.55819797

# Symmetrization protocol

In [9]:
from sp_frames_calculator import SPFramesCalculator
from pet_sp import PETSP
sp_hypers = Hypers()
sp_hypers.load_from_file(SP_HYPERS_PATH)
sp_frames_calculator = SPFramesCalculator(sp_hypers)

/home/sergey/final_reproduce/pet_latest/sp_frames_calculator.py:126: ComplexWarning: Casting complex values to real discards the imaginary part
  self.lambert_constant = torch.tensor(float(lambertw(np.exp(-1.0))))


# symmetrization of normal model

In [10]:
model_sp = PETSP(model, None,
                 100.0, hypers.USE_ENERGIES, hypers.USE_FORCES,
                 sp_frames_calculator, 50,
                 epsilon = EPSILON).cuda()

for _ in range(10):
    compute_for_random_rotation(model_sp, is_sp = True)

tensor([ 9.5778741837,  3.8056445122, 24.8481578827,  5.5652675629],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 9.5778713226,  3.8056442738, 24.8481521606,  5.5652666092],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 9.5778722763,  3.8056440353, 24.8481521606,  5.5652675629],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 9.5778722763,  3.8056437969, 24.8481578827,  5.5652680397],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 9.5778722763,  3.8056442738, 24.8481502533,  5.5652670860],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 9.5778722763,  3.8056445122, 24.8481559753,  5.5652666092],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 9.5778722763,  3.8056437969, 24.8481540680,  5.5652675629],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 9.5778722763,  3.8056442738, 24.8481521606,  5.5652670860],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 9.5778722763,  3.8056437969, 24.8481540680,  5.56526756

# model combined of normal one and only length as auxiliary

In [11]:
model_sp = PETSP(model, model_only_length,
                 100.0, hypers.USE_ENERGIES, hypers.USE_FORCES,
                 sp_frames_calculator, 50,
                 epsilon = EPSILON).cuda()

for _ in range(10):
    compute_for_random_rotation(model_sp, is_sp = True)

tensor([ 9.5778713226,  3.8056442738, 24.8481559753,  5.5652680397],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 9.5778713226,  3.8056440353, 24.8481407166,  5.5652675629],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 9.5778722763,  3.8056445122, 24.8481502533,  5.5652680397],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 9.5778732300,  3.8056442738, 24.8481540680,  5.5652675629],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 9.5778732300,  3.8056442738, 24.8481502533,  5.5652680397],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 9.5778722763,  3.8056445122, 24.8481559753,  5.5652694702],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 9.5778713226,  3.8056437969, 24.8481616974,  5.5652656555],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 9.5778713226,  3.8056447506, 24.8481559753,  5.5652670860],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 9.5778722763,  3.8056435585, 24.8481464386,  5.56526756